### Python3 code to parse fmriprep confounds file (v1.5.8), and plot motion summary
Jamil Bhanji<br>
bhanji@psychology.rutgers.edu<br>
Apr 13, 2021<br>

### Step 1: Edit the variables in the cell below and run the cell.
* specify preprocessed data directory path
* specify path format to the fmriprep confounds file and output file, subject list, task names, run labels
* specify True/False to include 1 column for every outlier (use to regress out spikes)
    * specify what metrics to use to determine outliers (must be a column name in fmriprep confounds file 
        e.g. ["non-stdDVARS"] - )  

In [ ]:
###################### EDIT BELOW to change study specific paths and params #################3
preproctopdir = "/mnt/delgadolab/jamil/opiod_mita/preproc2/fmriprep" #study directory (full path)
#you can include "ses-01" or "ses-02" in the paths below and run it for one session at a time
#eg: confound_file_format = "/sub-%s/ses-01/func/sub-%s_task-%s_run-%s_bold_confounds.tsv"
#path should start from the top of the preproc folder (preproctopdir)
#there should be 4 "%s" placeholders that will be filled in later with subject, subject, task, run
confound_file_format = "/sub-%s/func/sub-%s_task-%s_run-%s_desc-confounds_regressors.tsv" 
output_file_format = "/sub-%s/func/sub-%s_task-%s_run-%s_desc-confounds_64plus.tsv" #this is the name of the motion confounds file that will be created
subjectlist =  ['601','603','604','606','607','608','609','610','611','612','613','614','615','616','617','618','622','624','625','626','627','629','630',
                   '801','802','803','804','806','807','809','810','811','812','813','814','815','816','817','818','819','820','821','822','823']

# it works for multiple tasks, multiple runs (see further below) -- but doesn't work for multiple sessions
func_tasks = ["cardtask"]#,"persist"]
func_task_runs = { 
    "cardtask": ("01",)#,
    #"persist": ("01",)#after each taskname (must match BIDS taskname) include run labels as list
} #comma inside parenthesis is important if there is only one run e.g.: ("01",)

####multiple tasks, multiple runs example:
# func_tasks = ["cardtask","persist"]
# func_task_runs = { 
#      "cardtask": ("01","02"), #after each taskname (must match BIDS taskname) include run labels as list
#      "persist": ("01",)  #comma after single item is important if there is only one run
# }
######

# set to True to add extra outliers to confound output file (e.g., fsl dvars, or framewise displacement) using boxplot thresh
include_outliers = False
#specify column names below for columns where you want to use the boxplot thresh method
outlier_metric_column_names = ["dvars"] #it's no longer possible to use multiple outlier metrics (like commented line below)
#outlier_metric_column_names = ["dvars","framewise_displacement"]


### Step 2: Run the cell below to create 24 column motion files and plot motion (framewise displacement)
* if you want to output something different then edit the function "fmriprep_confounds_to_24col_plusoutliers"

In [ ]:
# extract and plot
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#plotting style specs and display options
sns.set(rc={"axes.labelsize": 12})
#sns.set_style("white")
#sns.set_style("ticks")
projpalette = "muted" #main palette
%matplotlib inline

def square_and_diff(df):
    col_idxs = [df.columns.get_loc(c) for c in df.columns]
    for icol in col_idxs:
        colsq = df.iloc[:,icol]*df.iloc[:,icol]
        coldiff = df.iloc[:,icol].diff()
        df = pd.concat([df, colsq], axis=1)
        df = pd.concat([df, coldiff], axis=1)
        df = pd.concat([df, coldiff*coldiff], axis=1)
    return df

def make_outlier_columns(crit_data,metric):
    upperquartile = crit_data.quantile(.75)
    lowerquartile = crit_data.quantile(.25)
    outlier_thresh = crit_data.quantile(.75) + 1.5*(upperquartile-lowerquartile)
    outliers = crit_data.index[crit_data > outlier_thresh].tolist()
    outlier_df = pd.DataFrame(index=crit_data.index)
    for counter, outlier_idx in enumerate(outliers):
        colname = "%s%03d" % (metric,counter)
        outlier_df[colname] = 0
        outlier_df.iloc[outlier_idx,outlier_df.columns.get_loc(colname)] = 1
    return outlier_df

def fmriprep_confounds_select(infile, outfile, include_outliers, outlier_metric_column_names):
    """
    Read *_confounds.tsv, select motion, motion^2, deriv, deriv^2
    new Apr 2021: select first 5 a_comp_cor WM and first 5 a_comp_cor CSF 
      (recommendation from Ravi Mill, model 9 in Ciric et al 2017, https://doi.org/10.1016/j.neuroimage.2017.03.020)
    Output a tab separated file, BIDS compliant
    """
    #outdf = pd.DataFrame(columns=['onset','duration','trial_type','response_time'])
    if not(os.path.isfile(infile)):
        #print("no file: %s" % infile)
        return pd.DataFrame({'A' : []}), pd.DataFrame({'B' : []})  #return empty dataframe
    allconf_df = pd.read_csv(infile, sep='\t')
    outdf = allconf_df[['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z',
                        'trans_x_derivative1','trans_y_derivative1','trans_z_derivative1',
                        'rot_x_derivative1','rot_y_derivative1','rot_z_derivative1',
                        'trans_x_power2','trans_y_power2','trans_z_power2',
                        'rot_x_power2','rot_y_power2','rot_z_power2',
                        'trans_x_derivative1_power2','trans_y_derivative1_power2','trans_z_derivative1_power2',
                        'rot_x_derivative1_power2','rot_y_derivative1_power2','rot_z_derivative1_power2']].copy()
    #why these columns? -- see https://doi.org/10.1016/j.neuroimage.2017.03.020 (Ciric et al 2017, NeuroImage)
    #consider 'a_comp_cor_XX' columns (e.g., first 5) if not using the aroma-denoised file
    #notes from pilot scan -- contrasts look similar with/without aroma, with/without acompcor, but slightly better
    #if we don't use aroma and don't use acompcor. this doesn't mean much based on 1 scan.
    # add non steady state outliers if any (usually just first volume)
    if allconf_df.filter(regex="non_steady_state_outliers*").shape[1] > 0:
        outdf = pd.concat([outdf, allconf_df.filter(regex="non_steady_state_outliers*")], axis=1)
    # add 5 acompcor WM, 5 acompcor CSF, then add expanded (deriv, square, deriv squared)
    # first get the column names from the run specific json file    
    colinfo = pd.read_json(infile.replace("tsv","json"))
    WMcols = colinfo.loc[:,colinfo.loc['Mask',:]=='WM'].columns[:5]
    CSFcols = colinfo.loc[:,colinfo.loc['Mask',:]=='CSF'].columns[:5]
    allcols = WMcols.append(CSFcols)
    #now get expanded columns and append to out_df
    wm_csf_df = square_and_diff(allconf_df[allcols])
    outdf = pd.concat([outdf, wm_csf_df], axis=1)
    #outdf = square_and_diff(outdf) #square and deriv are in new versions of fmriprep confounds so we don't need this
    if include_outliers:
        outliers = pd.DataFrame(0,index = allconf_df.index, columns = ['outliers'])
        #removed option to have multiple outlier metrics (bc it didn't handle overlapping outliers correctly, and bc
        #we don't use it anymore)
        metric = outlier_metric_column_names[0]
        outlier_cols = make_outlier_columns(allconf_df[metric],metric)
        outdf = pd.concat([outdf, outlier_cols], axis=1)
        outliers['outliers'] = outliers['outliers'] + outlier_cols.sum(axis=1)
        #specify column names below to include outlier columns that are provided by fmriprep
        #not implemented yet
    else:
        outliers = pd.DataFrame(0,index = allconf_df.index, columns = ['outliers'])
    outliers.loc[outliers['outliers']>1,'outliers'] = 1
    outdf = outdf.round(10)
    outdf.fillna(0).to_csv(outfile, sep='\t', float_format='%.10f', index=False, header=False)
    #print(outdf.fillna(0).head())
    return allconf_df.framewise_displacement, outliers

total_plots = 0
for isub, subject in enumerate(subjectlist):
    for itask, task in enumerate(func_tasks):
        total_plots=total_plots+len(func_task_runs[task])
        
fig, axarr = plt.subplots(nrows=total_plots, ncols=2, sharey=False, sharex=False, constrained_layout=True) #requires matplotlib>v3.1
fig.set_figwidth(16)
fig.set_figheight(4*total_plots)
#fig.subplots_adjust(hspace=.3)
fig.suptitle("Framewise Displacement and Outliers (%s >1.5IQR above 75th percentile)" %(",".join(outlier_metric_column_names)), fontsize=22,
             fontweight="bold", color="black")#, y= .98, verticalalignment="bottom")

fd_data = pd.DataFrame(columns=['subject','task','run','displacement'])
outlier_data = pd.DataFrame(columns=['subject','task','run','outliers'])
plotnum=0
for isub, subject in enumerate(subjectlist):
    for itask, task in enumerate(func_tasks):
        for irun, run in enumerate(func_task_runs[task]):
            if total_plots==1:
                ax = axarr[0]
                textax=axarr[1]
            else:
                ax = axarr[plotnum,0]
                textax=axarr[plotnum,1]
            textax.axis("off")
            plotnum=plotnum+1
            confoundinfile = (preproctopdir + confound_file_format % (subject,subject,task,run))
            outfile24 = (preproctopdir + output_file_format % (subject,subject,task,run))
            displacement, outliers = fmriprep_confounds_select(confoundinfile, 
                                                               outfile24, 
                                                               include_outliers,
                                                               outlier_metric_column_names)
            if displacement.empty:
                subdf = pd.DataFrame({'subject': subject, 'task': task, 'run': run, 'displacement': 0}, index=range(100))
                suboutlierdf = pd.DataFrame({'subject': subject, 'task': task, 'run': run, 'outliers': 0}, index=range(100))
                runlabel = "no run %s data" % run
                ax.text(0,.5, "no data for  %s" %(confoundinfile), color="red", fontweight="bold", fontsize=12)
            else:
                subdf = pd.DataFrame({'subject': subject, 'task': task, 'run': run, 'displacement': displacement.values})
                suboutlierdf = pd.DataFrame({'subject': subject, 'task': task, 'run': run, 'outliers': outliers['outliers'].values}, 
                                            index=range(len(outliers.values)))
                runlabel = "run %s" % run
                #all subjects motion data is saved in one dataframe, fd_data, in case you want to export it
                fd_data = fd_data.append(subdf, ignore_index=True)
                outlier_data = outlier_data.append(suboutlierdf, ignore_index=True)
                ax.plot(fd_data.loc[(fd_data['subject']==subject) & 
                                    (fd_data['task']==task) & 
                                    (fd_data['run']==run)]['displacement'].fillna(0).values, label=runlabel)
                outlier_plot_vals = np.ma.array(outlier_data.loc[(outlier_data['subject']==subject) &
                                                                 (outlier_data['task']==task) & 
                                                                 (outlier_data['run']==run)]['outliers'].fillna(0).values)
                outlier_plot_x = range(len(outlier_plot_vals))
                outlier_plot_vals = np.ma.masked_where(outlier_plot_vals < 1 , outlier_plot_vals)
                ax.plot(outlier_plot_x, outlier_plot_vals*fd_data.loc[(fd_data['subject']==subject) & 
                                                                      (fd_data['task']==task) & 
                                                                      (fd_data['run']==run)]['displacement'].max(),
                        "o", label=runlabel+"-outlier")
                ax.set_title("subject %s task %s" % (subject,task),fontweight="bold")
                ax.legend(loc='best')
                if include_outliers:
                    sidetext = "%d outliers" %(suboutlierdf['outliers'].sum())
                else:
                    sidetext = "no outlier check"
                textax.text(0,.5, sidetext, color="red", fontsize=12)
                    


### Errors?
* Make sure you have write permission in the fmriprep directory (see fmriprep tips on delgadolab google drive)
* Make sure the list of tasks and runs is specified correctly (see comments in 1st code cell) 
* Double check the file paths in first code cell

### Step 3: Add the new confound file to your 1st level glm analysis
* e.g., in FSL's Feat, select "Add additional confound EVs" and enter the filename for the newly generated motion confound files (e.g., sub-101_task-cardtask_run-01_bold_confounds24plusoutliers.tsv)
* in SPM you can specify it as a user-defined covariate (I think that's what it's called)

### Step 4: Export the notebook to html to save the plots and code in a nice portable format
* use the notebook's file menu-> Download As -> HTML